<a href="https://colab.research.google.com/github/Blvisse/DataExtraction/blob/master/Formula1_WebScrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
import bs4 as bs
from requests import get
from urllib.request import urlopen
import pandas as pd

In [3]:
source=urlopen("https://www.formula1.com/en/results.html/1950/races/94/great-britain/race-result.html")
soup=bs.BeautifulSoup(source,'lxml')
table=soup.find('table')
data=pd.read_html(str(table),flavor='bs4', header=[0])[0]
data.drop(["Unnamed: 0","Unnamed: 8"],axis=1,inplace=True)


In [4]:
data

,Pos,No,Driver,Car,Laps,Time/Retired,PTS
0,1,2,Nino Farina FAR,Alfa Romeo,70.0,2:13:23.600,9
1,2,3,Luigi Fagioli FAG,Alfa Romeo,70.0,+2.600s,6
2,3,4,Reg Parnell PAR,Alfa Romeo,70.0,+52.000s,4
3,4,14,Yves Giraud Cabantous CAB,Talbot-Lago,68.0,+2 laps,3
4,5,15,Louis Rosier ROS,Talbot-Lago,68.0,+2 laps,2
5,6,12,Bob Gerard GER,ERA,67.0,+3 laps,0
6,7,11,Cuth Harrison HAR,ERA,67.0,+3 laps,0
7,8,16,Philippe Etancelin ETA,Talbot-Lago,65.0,+5 laps,0
8,9,6,David Hampshire HAM,Maserati,64.0,+6 laps,0
9,10,10,Joe Fry FRY,Maserati,NaN,SHC,0


In [5]:
HOMEPAGE = 'https://www.formula1.com/'
YEAR = '1990'
race_result='race-result.html'

In [6]:
def get_race_urls(year):
    race_urls = []
    source =urlopen(f"https://www.formula1.com/en/results.html/{YEAR}/races.html").read()
    soup=bs.BeautifulSoup(source,'lxml')
    
    
    
    for url in soup.find_all('a'):
        if YEAR in str(url.get('href')) and 'race-result' in str(url.get('href'))and url.get('href') not in race_urls:
                                        
            race_urls.append(url.get('href'))
    return race_urls

In [7]:
race_urls=get_race_urls(YEAR)

In [8]:
race_urls

['/en/results.html/1990/races/64/united-states/race-result.html',
 '/en/results.html/1990/races/65/brazil/race-result.html',
 '/en/results.html/1990/races/66/san-marino/race-result.html',
 '/en/results.html/1990/races/67/monaco/race-result.html',
 '/en/results.html/1990/races/68/canada/race-result.html',
 '/en/results.html/1990/races/69/mexico/race-result.html',
 '/en/results.html/1990/races/70/france/race-result.html',
 '/en/results.html/1990/races/71/great-britain/race-result.html',
 '/en/results.html/1990/races/72/germany/race-result.html',
 '/en/results.html/1990/races/73/hungary/race-result.html',
 '/en/results.html/1990/races/74/belgium/race-result.html',
 '/en/results.html/1990/races/75/italy/race-result.html',
 '/en/results.html/1990/races/76/portugal/race-result.html',
 '/en/results.html/1990/races/77/spain/race-result.html',
 '/en/results.html/1990/races/78/japan/race-result.html',
 '/en/results.html/1990/races/79/australia/race-result.html']

In [9]:

season_results_df = None


def seasons_results(race_urls):

    for n, race in enumerate(race_urls):

        placeholder = [0 for i in range(n)]
        race_name = race.split('/')[6]

        results_page = urlopen(f"{HOMEPAGE}{race}").read()
        race_results =bs.BeautifulSoup(results_page,'lxml')

        table = race_results.find_all('table')[0] 
        df = pd.read_html(str(table), flavor='bs4', header=[0])[0]
        df.drop(["Unnamed: 0","Unnamed: 8"], axis=1, inplace=True)
        df.set_index('No', inplace=True)

        #establish season results df on first race information
        if n == 0:
            season_results_df = pd.DataFrame(df[['Driver','Car']], columns=['Driver','Car'], index=df.index)
       
        #add drivers if they are not in season_results_df
        for ind in df.index.difference(season_results_df.index):
            season_results_df.loc[ind] = [df['Driver'].loc[ind],df['Car'].loc[ind],*placeholder]

        for ind in df.index:
            pts = df['PTS'].where(df.index == ind).dropna().values[0]
#             if pts == 9:
#                 pts = 10 #if mapping differnt points systems
            season_results_df.loc[ind, race_name] = int(pts)
    
    #####Format the dataframe with a few lines#####
    season_results_df.sort_index(inplace=True)
    season_results_df.fillna(0, inplace=True)   
    season_results_df['Car'] = season_results_df['Car'].apply(lambda s : s[:3]).map(str.upper) #retain last 3 digits and caps
    return season_results_df

In [ ]:
season_results_df = seasons_results(race_urls)
season_results_df


In [11]:
filename='DriverList.csv'
season_results_df.to_csv(filename,index=False)